In [ ]:
import openai
import pandas as pd
import time

In [ ]:
# 🔐 Set your OpenAI API key
# openai.api_key = ""  # Replace with your real OpenAI API key
# 🔐 Set your OpenAI API key
client = openai.OpenAI(api_key="<<ADD Key>>")  # Replace with your key

In [ ]:
# 📄 Load emotion similarity scores (CLAP output)
# csv_path = "/content/split_equal_part_2.csv"
csv_path = "Path to .csv file from Algorithm 1 (CLAP Annotations)"
df = pd.read_csv(csv_path)

In [ ]:
# 🧠 Build prompt from top-3 emotion tags
def build_prompt(file_row):
    top_scores = file_row[1:].sort_values(ascending=False)
    top_emotions = ", ".join([f"{k} ({v:.2f})" for k, v in top_scores.head(3).items()])
    prompt = (
        f"Based on the following top 3 emotional tags for a song — {top_emotions} — generate a brief and accurate description of the song "
        f"suitable for use in a music therapy session for children with autism spectrum disorder (ASD).\n\n"
        "The description should focus on the song’s emotional qualities, its musical characteristics, and how it might support therapeutic goals "
        "such as calming, attention, engagement, or sensory regulation.\n\n"
        "Use language that is professional but easy to understand for therapists and educators. Limit the description to 2–3 sentences."
    )
    return prompt

In [ ]:
# ✍️ Generate metadata
descriptions = []

for idx, row in df.iterrows():
    try:
        prompt = build_prompt(row)
        print(f"🔎 Prompting GPT for {row['file_name']}...")

        response = client.chat.completions.create(
            model="gpt-4o",  # or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are a helpful assistant trained in music therapy for children with ASD."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=3000
        )

        description = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ Failed for {row['file_name']}: {e}")
        description = ""

    descriptions.append(description)
    time.sleep(1.1)  # Optional: rate limit buffer

In [ ]:
# 💾 Save output CSV
df_out = pd.DataFrame({
    "file_name": df["file_name"],
    "description": descriptions
})
df_out.to_csv("clap_combined.csv", index=False)
print("✅ Metadata generation complete.")